#! pip install pytesseract
Step 1: Detect edges.
Step 2: Use the edges in the image to find the contour (outline) representing the piece of paper being scanned.
Step 3: Apply a perspective transform to obtain the top-down view of the document.

In [2]:
from PIL import Image
import pytesseract

In [3]:
#im= Image.open("Medical_History.jpg")

In [4]:
#text = pytesseract.image_to_string(im, lang = 'eng')

In [5]:
#print(text)

In [6]:
#!pip install opencv-python

In [7]:
import cv2
import numpy as np
#!pip install --upgrade imutils


In [8]:
img = cv2.imread("Walmart.10.jpg")
#cv2.imshow('Original Image', img) 

In [9]:
print(type(img))

<class 'numpy.ndarray'>


In [10]:
imS = cv2.resize(img, (500, 500))                    
cv2.imshow('Original Image', imS) 
cv2.waitKey(0)

-1

In [11]:
print(imS.shape[0:2])
print(img.shape[0:2])

(500, 500)
(3264, 2448)


In [12]:
height, width = img.shape[0:2]

In [13]:
rotationMatrix = cv2.getRotationMatrix2D((width, height), 90, .5)

In [14]:
rotatedImage = cv2.warpAffine(img, rotationMatrix, (width, height))

In [15]:
cv2.imshow('Rotated Image', rotatedImage)
 
cv2.waitKey(0)

-1

In [16]:
import PIL

In [17]:
from pyimagesearch.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as npa
import argparse
import cv2
import imutils

In [29]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required = True,
   help = "Walmart.10.jpg.jpg")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -i IMAGE
ipykernel_launcher.py: error: the following arguments are required: -i/--image


SystemExit: 2

In [30]:
image = cv2.imread("Walmart.10.jpg")
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)


In [31]:
type(image)

numpy.ndarray

In [32]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

In [35]:
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 1: Edge Detection


In [36]:
screenCnt=None
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
      screenCnt = approx
      break
 
# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 2: Find contours of paper


error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\drawing.cpp:2606: error: (-215:Assertion failed) reader.ptr != NULL in function 'cvDrawContours'


In [67]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
 
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
 
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

STEP 3: Apply perspective transform


-1

In [68]:
type(warped)
image = cv2.cvtColor(warped,cv2.COLOR_GRAY2RGB)

In [69]:
from pyimagesearch.transform import four_point_transform
import numpy as np
import argparse
import cv2


In [70]:
import argparse
import cv2

In [97]:
def decode_predictions(scores, geometry):
    # grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding
    # confidence scores
        (numRows, numCols) = scores.shape[2:4]
        rects = []
        confidences = []

        # loop over the number of rows
        for y in range(0, numRows):
                # extract the scores (probabilities), followed by the
                # geometrical data used to derive potential bounding box
                # coordinates that surround text
                scoresData = scores[0, 0, y]
                xData0 = geometry[0, 0, y]
                xData1 = geometry[0, 1, y]
                xData2 = geometry[0, 2, y]
                xData3 = geometry[0, 3, y]
                anglesData = geometry[0, 4, y]

                # loop over the number of columns
                for x in range(0, numCols):
                        # if our score does not have sufficient probability,
                        # ignore it
                        if scoresData[x] < 0.01 :
                                continue

                        # compute the offset factor as our resulting feature
                        # maps will be 4x smaller than the input image
                        (offsetX, offsetY) = (x * 4.0, y * 4.0)

                        # extract the rotation angle for the prediction and
                        # then compute the sin and cosine
                        angle = anglesData[x]
                        cos = np.cos(angle)
                        sin = np.sin(angle)

                        # use the geometry volume to derive the width and height
                        # of the bounding box
                        h = xData0[x] + xData2[x]
                        w = xData1[x] + xData3[x]

                        # compute both the starting and ending (x, y)-coordinates
                        # for the text prediction bounding box
                        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
                        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
                        startX = int(endX - w)
                        startY = int(endY - h)

                        # add the bounding box coordinates and probability score
                        # to our respective lists
                        rects.append((startX, startY, endX, endY))
                        confidences.append(scoresData[x])

        # return a tuple of the bounding boxes and associated confidences
        return (rects, confidences)


In [98]:
orig = image.copy()
(H, W) = image.shape[:2]
(origH, origW) = image.shape[:2]

In [99]:
(newW, newH) = (640, 320)
rW = origW / float(newW)
rH = origH / float(newH)

In [100]:
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

In [101]:
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

In [102]:
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet("frozen_east_text_detection.pb")

[INFO] loading EAST text detector...


In [103]:
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    (123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

In [18]:
print((scores, geometry))

NameError: name 'geometry' is not defined

In [1]:
from imutils.object_detection import non_max_suppression
(rects, confidences) = decode_predictions(scores, geometry)
boxes = non_max_suppression(np.array(rects), probs=confidences)
print((scores))

NameError: name 'decode_predictions' is not defined

In [106]:
results = []

In [107]:
for (startX, startY, endX, endY) in boxes:
         # scale the bounding box coordinates based on the respective
         # ratios
         startX = int(startX * rW)
         startY = int(startY * rH)
         endX = int(endX * rW)
         endY = int(endY * rH)

         # in order to obtain a better OCR of the text we can potentially
         # apply a bit of padding surrounding the bounding box -- here we
         # are computing the deltas in both the x and y directions
         dX = int((endX - startX) * 1)
         dY = int((endY - startY) * 1)

         # apply padding to each side of the bounding box, respectively
         startX = max(0, startX - dX)
         startY = max(0, startY - dY)
         endX = min(origW, endX + (dX * 2))
         endY = min(origH, endY + (dY * 2))

         # extract the actual padded ROI
         roi = orig[startY:endY, startX:endX]

         # in order to apply Tesseract v4 to OCR text we must supply
         # (1) a language, (2) an OEM flag of 4, indicating that the we
         # wish to use the LSTM neural net model for OCR, and finally
         # (3) an OEM value, in this case, 7 which implies that we are
         # treating the ROI as a single line of text
         config = ("-l eng --oem 1 --psm 7")
         text = pytesseract.image_to_string(roi, config=config)

         # add the bounding box coordinates and OCR'd text to the list
         # of results
         results.append(((startX, startY, endX, endY), text))
         print("==================")


In [108]:
print(results)

[]


In [109]:
results = sorted(results, key=lambda r:r[0][1])

# loop over the results
for ((startX, startY, endX, endY), text) in results:
         # display the text OCR'd by Tesseract
         print("OCR TEXT")
         print("========")
         print("{}\n".format(text))

         # strip out non-ASCII text so we can draw the text on the image
         # using OpenCV, then draw the text and a bounding box surrounding
         # the text region of the input image
         text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
         output = orig.copy()
         cv2.rectangle(output, (startX, startY), (endX, endY),
                  (0, 0, 255), 2)
         cv2.putText(output, text, (startX, startY - 20),
                  cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

         

In [110]:
# show the output image
cv2.imshow("Text Detection", output)
cv2.waitKey(0)

NameError: name 'output' is not defined